In [1]:
import pathlib
import random

import dalex as dx
import numpy as np
import pandas as pd
import seaborn as sns
import xgboost as xgb
from scipy.stats import spearmanr
from sklearn import model_selection, ensemble

sns.set_theme()

C:\Users\alan\anaconda3\envs\xai\lib\site-packages\xgboost\compat.py:36: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import MultiIndex, Int64Index


In [2]:
def seed_everything(seed: int) -> int:
    random.seed(seed)
    np.random.seed(seed)

    return seed

In [3]:
SEED = 357
seed_everything(SEED)

357

In [4]:
data = pd.read_csv('../data/raw/temat_2_dane.csv')
data_cols_description = pd.read_excel('../data/raw/temat_2_opis_zmiennych.xlsx')

In [5]:
name_to_description = {}
for x, y in zip(data_cols_description['NAZWA'], data_cols_description['OPIS']):
    name_to_description[x] = y

In [6]:
def train_test_split_by_income(data, test_size=0.2, bins=5):
    income_bined = pd.qcut(data['Y'], q=bins)

    return model_selection.train_test_split(data, test_size=test_size, shuffle=True, stratify=income_bined, random_state=SEED)

In [7]:
data_train_val, data_test = train_test_split_by_income(data)
data_train, data_val = train_test_split_by_income(data_train_val, test_size=0.25)

In [8]:
X_train, y_train = data_train.drop('Y', axis=1), data_train['Y']
X_val, y_val = data_val.drop('Y', axis=1), data_val['Y']

vars = ['X1'] + [f'X{j}' for j in range(4, 295 + 1, 7)]

X_train = X_train[vars]
X_val = X_val[vars]

for dataset in [X_train, y_train, X_val, y_val]:
    dataset.reset_index(drop=True, inplace=True)

In [9]:
xgb_best_params = {"max_depth": 23, "min_child_weight": 2.404712074481848, "colsample_bytree": 0.5095502611913834, "learning_rate": 0.0449682801162507, "subsample": 0.9196040393707718, "alpha": 0.0524693644585006, "lambda": 4.5586068865713215e-07}

In [10]:
model_xgb = xgb.XGBRegressor(verbosity=0, objective='reg:squarederror', n_estimators=10000, **xgb_best_params, random_state=SEED)
model_xgb.fit(X_train, np.log10(y_train + 1), eval_set=[(X_val, np.log10(y_val + 1))], eval_metric='mae', early_stopping_rounds=50)

[0]	validation_0-mae:3.18138
[1]	validation_0-mae:3.03873
[2]	validation_0-mae:2.90246
[3]	validation_0-mae:2.77228
[4]	validation_0-mae:2.64797
[5]	validation_0-mae:2.52929
[6]	validation_0-mae:2.41616
[7]	validation_0-mae:2.30791
[8]	validation_0-mae:2.20449
[9]	validation_0-mae:2.10574
[10]	validation_0-mae:2.01133
[11]	validation_0-mae:1.92114
[12]	validation_0-mae:1.83514
[13]	validation_0-mae:1.75264
[14]	validation_0-mae:1.67415
[15]	validation_0-mae:1.59924
[16]	validation_0-mae:1.52769
[17]	validation_0-mae:1.45930
[18]	validation_0-mae:1.39409
[19]	validation_0-mae:1.33185
[20]	validation_0-mae:1.27221
[21]	validation_0-mae:1.21528
[22]	validation_0-mae:1.16094
[23]	validation_0-mae:1.10921
[24]	validation_0-mae:1.05975
[25]	validation_0-mae:1.01281
[26]	validation_0-mae:0.96784
[27]	validation_0-mae:0.92473


C:\Users\alan\anaconda3\envs\xai\lib\site-packages\xgboost\data.py:250: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  elif isinstance(data.columns, (pd.Int64Index, pd.RangeIndex)):


[28]	validation_0-mae:0.88377
[29]	validation_0-mae:0.84452
[30]	validation_0-mae:0.80721
[31]	validation_0-mae:0.77189
[32]	validation_0-mae:0.73792
[33]	validation_0-mae:0.70557
[34]	validation_0-mae:0.67475
[35]	validation_0-mae:0.64534
[36]	validation_0-mae:0.61732
[37]	validation_0-mae:0.59092
[38]	validation_0-mae:0.56562
[39]	validation_0-mae:0.54161
[40]	validation_0-mae:0.51866
[41]	validation_0-mae:0.49699
[42]	validation_0-mae:0.47623
[43]	validation_0-mae:0.45671
[44]	validation_0-mae:0.43839
[45]	validation_0-mae:0.42100
[46]	validation_0-mae:0.40461
[47]	validation_0-mae:0.38908
[48]	validation_0-mae:0.37438
[49]	validation_0-mae:0.36061
[50]	validation_0-mae:0.34751
[51]	validation_0-mae:0.33525
[52]	validation_0-mae:0.32364
[53]	validation_0-mae:0.31297
[54]	validation_0-mae:0.30262
[55]	validation_0-mae:0.29308
[56]	validation_0-mae:0.28405
[57]	validation_0-mae:0.27553
[58]	validation_0-mae:0.26785
[59]	validation_0-mae:0.26062
[60]	validation_0-mae:0.25380
[61]	valid

XGBRegressor(alpha=0.0524693644585006, base_score=0.5, booster='gbtree',
             colsample_bylevel=1, colsample_bynode=1,
             colsample_bytree=0.5095502611913834, enable_categorical=False,
             gamma=0, gpu_id=-1, importance_type=None,
             interaction_constraints='', lambda=4.5586068865713215e-07,
             learning_rate=0.0449682801162507, max_delta_step=0, max_depth=23,
             min_child_weight=2.404712074481848, missing=nan,
             monotone_constraints='()', n_estimators=10000, n_jobs=12,
             num_parallel_tree=1, predictor='auto', random_state=357,
             reg_alpha=0.0524693653, reg_lambda=4.55860715e-07,
             scale_pos_weight=1, subsample=0.9196040393707718,
             tree_method='exact', validate_parameters=1, verbosity=0)

In [11]:
predict = lambda model, data: 10 ** model.predict(data) - 1

In [12]:
explainer_xgb = dx.Explainer(model_xgb, X_train.rename(name_to_description, axis=1), y_train, predict_function=predict, label='XGBoost')

Preparation of a new explainer is initiated

  -> data              : 6000 rows 43 cols
  -> target variable   : Parameter 'y' was a pandas.Series. Converted to a numpy.ndarray.
  -> target variable   : 6000 values
  -> model_class       : xgboost.sklearn.XGBRegressor (default)
  -> label             : XGBoost
  -> predict function  : <function <lambda> at 0x000002AD80BAC820> will be used
  -> predict function  : Accepts pandas.DataFrame and numpy.ndarray.
  -> predicted values  : min = 67.3, mean = 9.1e+03, max = 2.85e+05
  -> model type        : regression will be used (default)
  -> residual function : difference between y and yhat (default)
  -> residuals         : min = -7.56e+03, mean = 75.5, max = 3.38e+04
  -> model_info        : package xgboost

A new explainer has been created!


In [13]:
random_observations = data_val.sample(3, random_state=133)
X_random_observations = random_observations.drop('Y', axis=1)[vars]
y_random_observations = random_observations['Y']

In [14]:
for pred, true in zip(predict(model_xgb, X_random_observations), y_random_observations):
    print(f'pred: {pred} \t true: {true} \t abs residual: {abs(pred - true)}')

pred: 3421.353759765625 	 true: 3466.8 	 abs residual: 45.44624023437518
pred: 5373.76318359375 	 true: 5789.29 	 abs residual: 415.52681640624996
pred: 3292.744873046875 	 true: 3282.0 	 abs residual: 10.744873046875


In [15]:
for ith, (X_random, y_random) in enumerate(zip(X_random_observations, y_random_observations)):
    cp = explainer_xgb.predict_profile(X_random_observations.iloc[ith])

    folder_path = pathlib.Path(f'../reports/figures/3')
    folder_path.mkdir(parents=True, exist_ok=True)
    for i in range(len(X_train.columns)):
        plot = cp.plot(facet_ncol=1, variables=X_train.rename(name_to_description, axis=1).columns[i:i+1], show=False)
        plot.write_image(file=folder_path / f'{ith}_{i}.png', engine='orca', scale=2.0)

Calculating ceteris paribus: 100%|██████████| 43/43 [00:00<00:00, 184.74it/s]


In [16]:
i = 0
while True:
    samples = X_val.rename(name_to_description, axis=1).sample(2, random_state=SEED + i)

    cp_xgb_sample0 = explainer_xgb.predict_profile(samples.iloc[0])
    cp_xgb_sample1 = explainer_xgb.predict_profile(samples.iloc[1])

    age_sample0 = cp_xgb_sample0.result[cp_xgb_sample0.result['_vname_'] == 'wiek klienta'][['wiek klienta', '_yhat_']]
    age_sample1 = cp_xgb_sample1.result[cp_xgb_sample1.result['_vname_'] == 'wiek klienta'][['wiek klienta', '_yhat_']]

    len_sample0 = len(age_sample0['_yhat_'])
    len_sample1 = len(age_sample1['_yhat_'])
    if len_sample0 < len_sample1:
        test = spearmanr(age_sample0['_yhat_'], age_sample1['_yhat_'][:len_sample0])
    elif len_sample0 > len_sample1:
        test = spearmanr(age_sample0['_yhat_'][:len_sample1], age_sample1['_yhat_'])
    else:
        test = spearmanr(age_sample0['_yhat_'], age_sample1['_yhat_'])

    corr, pvalue = test
    print(corr)
    if corr < -0.5 and pvalue < 0.05:
        break

    i += 1


Calculating ceteris paribus: 100%|██████████| 43/43 [00:00<00:00, 191.92it/s]


0.6085118027756611


Calculating ceteris paribus: 100%|██████████| 43/43 [00:00<00:00, 194.53it/s]


0.2198852370325739


Calculating ceteris paribus: 100%|██████████| 43/43 [00:00<00:00, 183.62it/s]


...

0.6887876331620765


Calculating ceteris paribus: 100%|██████████| 43/43 [00:00<00:00, 199.23it/s]


0.828686696898416


Calculating ceteris paribus:  72%|███████▏  | 31/43 [00:00<00:00, 190.14it/s]


KeyboardInterrupt: 

In [ ]:
folder_path = pathlib.Path(f'../reports/figures/4')
folder_path.mkdir(parents=True, exist_ok=True)

for ith, sample_explanation in enumerate([cp_xgb_sample0, cp_xgb_sample1]):
    for i in range(len(X_train.columns)):
        plot = sample_explanation.plot(facet_ncol=1, variables=X_train.rename(name_to_description, axis=1).columns[i:i+1], show=False)
        plot.write_image(file=folder_path / f'{ith}_{int(i)}.png', engine='orca', scale=2.0)

In [17]:
rf_best_params = {"n_estimators": 750, "max_features": 0.2969145621920279}
model_rf = ensemble.RandomForestRegressor(**rf_best_params, random_state=SEED)
model_rf.fit(X_train, np.log10(y_train + 1))

RandomForestRegressor(max_features=0.2969145621920279, n_estimators=750,
                      random_state=357)

In [18]:
explainer_rf = dx.Explainer(model_rf, X_train.rename(name_to_description, axis=1), y_train, predict_function=predict, label='RandomForest')

Preparation of a new explainer is initiated

  -> data              : 6000 rows 43 cols
  -> target variable   : Parameter 'y' was a pandas.Series. Converted to a numpy.ndarray.
  -> target variable   : 6000 values
  -> model_class       : sklearn.ensemble._forest.RandomForestRegressor (default)
  -> label             : RandomForest
  -> predict function  : <function <lambda> at 0x000002AD80BAC820> will be used
  -> predict function  : Accepts pandas.DataFrame and numpy.ndarray.


C:\Users\alan\anaconda3\envs\xai\lib\site-packages\sklearn\base.py:450: UserWarning:

X does not have valid feature names, but RandomForestRegressor was fitted with feature names

C:\Users\alan\anaconda3\envs\xai\lib\site-packages\sklearn\base.py:493: FutureWarning:

The feature names should match those that were passed during fit. Starting version 1.2, an error will be raised.
Feature names unseen at fit time:
- Sumaryczna kwota transakcji w bankomatach w 6 ostatnich miesiącach
- Sumaryczna kwota wpływów klienta w w 6 ostatnich miesiącach
- Sumaryczna kwota wpływów klienta z tytułu emerytury w w 6 ostatnich miesiącach
- Sumaryczna kwota wpływów klienta z tytułu programu 500 plus w w 6 ostatnich miesiącach
- Sumaryczna kwota wpływów klienta z tytułu stypendium w w 6 ostatnich miesiącach
- ...
Feature names seen at fit time, yet now missing:
- X1
- X102
- X109
- X11
- X116
- ...




  -> predicted values  : min = 3.37e+02, mean = 8.16e+03, max = 1.22e+05
  -> model type        : regression will be used (default)
  -> residual function : difference between y and yhat (default)


C:\Users\alan\anaconda3\envs\xai\lib\site-packages\sklearn\base.py:493: FutureWarning:

The feature names should match those that were passed during fit. Starting version 1.2, an error will be raised.
Feature names unseen at fit time:
- Sumaryczna kwota transakcji w bankomatach w 6 ostatnich miesiącach
- Sumaryczna kwota wpływów klienta w w 6 ostatnich miesiącach
- Sumaryczna kwota wpływów klienta z tytułu emerytury w w 6 ostatnich miesiącach
- Sumaryczna kwota wpływów klienta z tytułu programu 500 plus w w 6 ostatnich miesiącach
- Sumaryczna kwota wpływów klienta z tytułu stypendium w w 6 ostatnich miesiącach
- ...
Feature names seen at fit time, yet now missing:
- X1
- X102
- X109
- X11
- X116
- ...




  -> residuals         : min = -7.15e+03, mean = 1.01e+03, max = 1.87e+05
  -> model_info        : package sklearn

A new explainer has been created!


In [31]:
for sample in X_val.iterrows():
    cp_xgb = explainer_xgb.predict_profile(sample[1])
    cp_rf = explainer_rf.predict_profile(sample[1])

    found = False
    for var in X_train.rename(name_to_description, axis=1).columns:
        var_xgb = cp_xgb.result[cp_xgb.result['_vname_'] == var][[var, '_yhat_']]
        var_rf = cp_rf.result[cp_rf.result['_vname_'] == var][[var, '_yhat_']]

        corr, pvalue = spearmanr(var_xgb['_yhat_'], var_rf['_yhat_'])
        if corr < -0.8 and pvalue < 0.05 and (
                abs(max(var_xgb['_yhat_']) - min(var_xgb['_yhat_'])) > 500 or
                abs(max(var_rf['_yhat_']) - min(var_rf['_yhat_'])) > 500
        ):
            found = True
            break
    if found:
        break


Calculating ceteris paribus:   0%|          | 0/43 [00:00<?, ?it/s]C:\Users\alan\anaconda3\envs\xai\lib\site-packages\sklearn\base.py:493: FutureWarning:

The feature names should match those that were passed during fit. Starting version 1.2, an error will be raised.
Feature names unseen at fit time:
- Sumaryczna kwota transakcji w bankomatach w 6 ostatnich miesiącach
- Sumaryczna kwota wpływów klienta w w 6 ostatnich miesiącach
- Sumaryczna kwota wpływów klienta z tytułu emerytury w w 6 ostatnich miesiącach
- Sumaryczna kwota wpływów klienta z tytułu programu 500 plus w w 6 ostatnich miesiącach
- Sumaryczna kwota wpływów klienta z tytułu stypendium w w 6 ostatnich miesiącach
- ...
Feature names seen at fit time, yet now missing:
- X1
- X102
- X109
- X11
- X116
- ...


C:\Users\alan\anaconda3\envs\xai\lib\site-packages\sklearn\base.py:493: FutureWarning:

The feature names should match those that were passed during fit. Starting version 1.2, an error will be raised.
Feature names unsee

In [32]:
var

'Sumaryczna kwota wydatków klienta na paliwo w w 6 ostatnich miesiącach'

In [33]:
folder_path = pathlib.Path(f'../reports/figures/5')
folder_path.mkdir(parents=True, exist_ok=True)

for model_name, sample_explanation in [('xgb', cp_xgb), ('rf', cp_rf)]:
    for i in range(len(X_train.columns)):
        plot = sample_explanation.plot(facet_ncol=1, variables=X_train.rename(name_to_description, axis=1).columns[i:i+1], show=False)
        plot.write_image(file=folder_path / f'{model_name}_{int(i)}.png', engine='orca', scale=2.0)